# Analyze EEG data

This notebook will help you prepare the figures for your EEG lab report. Before running this notebook, export six different files from LabChart:

1. MATLAB (.mat) file of selected data (raw recording) for eyes open and closed. Note that the goal of this figure is to see alpha waves (we do not want to see your entire experiment!)
2. **PSD .txt file** for eyes open and closed
3. **Spectrogram .txt. file** for eyes open and closed.

![](https://media3.giphy.com/media/2seGQ9JQrHzPlNgqFY/giphy.gif?cid=ecf05e477lrmp9f3xph0dmyxg5lh5uz9vxazrpve9wtmadgs&rid=giphy.gif&ct=g)
<hr>

## Figure 1. Raw EEG recordings

<mark>To run this, you will need to upload an exported .mat file from LabChart into your DataHub. [Follow these instructions on how to export your data as a .mat file](https://docs.google.com/document/d/1nscGwk4ZoLGoE1HhwHiOv8XimzUQ80RTLfwSNeMRO6E/edit#heading=h.njfsdrg02rx).</mark>

**For this report, you should export a selection of data (showing alpha waves!) and only the recorded channel.**

After exporting your selected data as a .mat file, change the filename below.

In [ ]:
# Change the filename to EXACTLY match your file
filename = 'EEG_alpha.mat'

# Load the data
from scipy.io import loadmat
data = loadmat(filename)['data'][0]

print('Data imported.')

Below, we'll set the sampling rate. Ensure that the printed sampling rate matches the sampling rate of your recording!

In [ ]:
import numpy as np

# Get the sampling rate and create a time axis for our data
sampling_rate = 400

timestamps = (np.arange(0, len(data)) * (1.0 / sampling_rate))*1000

print('Data was sampled at', sampling_rate, 'Hz.')

Now, we can plot the data!

In [ ]:
# import plotting packages
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

# set up figure & plt
fig,ax = plt.subplots(figsize=(10,4))

plt.plot(timestamps,data)

# You may need to change the x label
plt.xlabel('Time (ms)') 

# You may need to change the y label
plt.ylabel('Voltage (V)') 

# This makes the axis labels print without scientific notation
ax.ticklabel_format(useOffset=False, style='plain') 

# You can uncomment the line below to restrict the x axis plotting -- for example, to Zoom into alpha waves
#plt.xlim([200,700])

plt.show()

## Figure 2. Your PSD data
First, follow the instructions in the lab manual to export your PSD plot as a .txt file and upload that file to DataHub.

In [ ]:
# Import necessary packages
import pandas as pd

# Change the filename below to EXACTLY match your file
psd_filename = 'EEGpsd.txt'

psd_data = pd.read_fwf(psd_filename)
psd_data = psd_data.iloc[:,0].str.split('\t', expand=True).astype('float')
psd_data.columns = ['Frequency','EEG']
psd_data.head()

In [ ]:
# import plotting packages
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

# set up figure
fig,ax = plt.subplots(figsize=(6,6))

frequency = psd_data['Frequency']
EEG = psd_data['EEG']

plt.plot(frequency,EEG)

# You may need to change the x label
plt.xlabel('X LABEL') 

# You may need to change the y label
plt.ylabel('Y LABEL') 

# This makes the axis labels print without scientific notation
#ax.ticklabel_format(useOffset=False, style='plain') 

# You can uncomment the line below to restrict the x axis plotting
#plt.xlim([200,700])

plt.show()

## Figure 3. Your spectrogram
The following code will plot your spectrogram data from LabChart. First, you need to export a <mark>spectrogram</mark> .txt file and upload it to your DataHub.

### Step 1. Load Data

In [ ]:
# Import necessary packages
import numpy as np
import pandas as pd

# CHANGE YOUR FILE NAME HERE
spectrum_filename = 'EEG_rawSpectrogram.txt'

# Read the data & clean it up
frequency_labels_df = pd.read_fwf(spectrum_filename,sep='\t',header=2,encoding = 'unicode_escape')
frequency_labels = frequency_labels_df.columns.str.split('\t')
frequency_labels = frequency_labels[0][1:]

spectrum_data_df = pd.read_fwf(spectrum_filename,sep='\t',header=3,encoding = 'unicode_escape')
spectrum_data_df = spectrum_data_df.iloc[:,0].str.split('\t', expand=True)
spectrum_time = list(spectrum_data_df.iloc[:,0])
spectrum_data_df = spectrum_data_df.iloc[:,1:].astype('float')
spectrum_data_df.columns = frequency_labels
spectrum_array = np.array(spectrum_data_df)
spectrum_array

### Step 2. Clean data.
We still need to clean up the data a little bit, since the frequency labels (in the 0th row) are also in here, along with the time labels in the first column. Let's save these frequency & time labels, and then only save the necessary rows of our dataset. We should be left with an array that is just the actual data.

In [ ]:
def get_sec(time_str):
    """Get seconds from time."""
    m, s = time_str.split(':')
    return int(m) * 60 + int(s)

spectrum_time_s = []

for idx,row in enumerate(spectrum_time):
    if ':' in row:
        spectrum_time_s.append(float(get_sec(row)))
    else:
        spectrum_time_s.append(float(row))
        
recording_length = spectrum_time_s[-1] - spectrum_time_s[0]

print('Frequency range is ' + str(frequency_labels[0]) + ' to ' + str(frequency_labels[-1]) + ' Hz.')
print('Recording length is ' + str(recording_length) + ' seconds.') 

### Step 3. Rotate & flip array and subselect frequencies to plot.
Next, let's rotate & flip the frequency array so that it matches our conventional spectrogram format (time on y axis, frequencies going up the x axis). We'll also subselect the frequencies we want, rather than plotting up to 200 Hz.

In [ ]:
# Set your max frequency below
max_frequency = 50

# Round our frequency values so we can find one that matches our max frequency
freq_rounded = np.round(np.float_(frequency_labels))
freq_IDs = np.where(freq_rounded == max_frequency)

# Create a new frequency axis
freq_short = freq_rounded[0:freq_IDs[0][0]]

# Cut down the spectrum to only our desired frequencies
spectrum_clean = spectrum_array[:,0:freq_short.shape[0]]

# Rotate the spectrum to match typical spectrogram format
spectrum_rotated = np.rot90(spectrum_clean[1:,:]) # skip the frequency row
freq_labels = np.flip(freq_short,axis=0)

print('Your data will plot with a maximum frequency of '+str(max_frequency))

### Step 4. Plot the data!
Depending on the power in your dataset, you may need to change the maximum power plotted on the color bar. To do so, change the variable "max_v".

In [ ]:
num_ticks = 10

yticks = np.linspace(0, len(freq_labels) - 1, num_ticks, dtype=np.int)
yticklabels = [freq_labels[idx] for idx in yticks]

xticks = np.linspace(0, len(spectrum_time_s)-1, num_ticks, dtype=np.int)
xticklabels = [round(spectrum_time_s[idt],2) for idt in xticks]

# use the line below to change the heatmap scale bar
max_v = 1e-11

print('Your data will plot with a maximum V^2 of '+str(max_v))

Aaaaaand plot!

In [ ]:
# import plotting packages
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

# Set up the figure
fig,ax = plt.subplots(1,1,figsize=(8, 5), dpi= 80, facecolor='w', edgecolor='k')

plt.pcolor(np.flip(spectrum_rotated),vmax=max_v)

# Fix axes
ax.set_yticks(yticks)
ax.set_yticklabels([freq_short[idx] for idx in yticks])

ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels)

ax.set_ylabel('Frequency (Hz)',fontdict={'fontsize':14})
ax.set_xlabel('Time (seconds)',fontdict={'fontsize':14})

cbar = plt.colorbar()
cbar.ax.set_ylabel('Power ($V^2$)')


plt.show()

### Step 5. Right click on the image above to save it for your lab report.

If you want to run this code again on a different dataset (e.g., for eyes open instead of closed), we need to clear all of the variables. Run the reset code block below to do so. It will also prompt you to make sure you want to proceed. Type y and press enter.

In [ ]:
%reset